## <font color='green'>S&P500 Treemap</font>

#### Macbook need this to open NLTK

In [1]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

2024-06-29 11:41:26.285 Python[44927:9180284] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

#### Load necessary libraries

In [2]:
# libraries for webscraping, parsing and getting stock data
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import yfinance as yf

# for plotting and data manipulation
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly
import plotly.express as px

# NLTK VADER for sentiment analysis
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
from datetime import datetime
import warnings
import requests

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/yimingniu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# From Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'id': 'constituents'})

# Parse the table
df = pd.read_html(str(table))[0]
df = df[['Symbol', 'Security', 'GICS Sector']]
df.columns = ['Ticker', 'Company', 'Sector']

df.head()


,Ticker,Company,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott,Health Care
3,ABBV,AbbVie,Health Care
4,ACN,Accenture,Information Technology


### <font color='green'>Scrape the Date, Time and News Headlines Data (Some error when reading the tickers)</font>

In [4]:
# Scrape the Date, Time and News Headlines Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

tickers = df['Ticker'].tolist()

for ticker in tickers:
    print(ticker)
    url = finwiz_url + ticker
    req = Request(url=url,headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}) 
    try:
        response = urlopen(req)    
        # Read the contents of the file into 'html'
        html = BeautifulSoup(response, 'html.parser')
        # Find 'news-table' in the Soup and load it into 'news_table'
        news_table = html.find(id='news-table')
        # Add the table to our dictionary
        news_tables[ticker] = news_table
        # Avoid HTTP Error 429
        time.sleep(0.3)
    except Exception as e:
        print("Error")
        continue

MMM
AOS
ABT
ABBV
ACN
ADBE
AMD
AES
AFL
A
APD
ABNB
AKAM
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
AME
AMGN
APH
ADI
ANSS
AON
APA
AAPL
AMAT
APTV
ACGL
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BK
BBWI
BAX
BDX
BRK.B
Error
BBY
BIO
TECH
BIIB
BLK
BX
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BRO
BF.B
Error
BLDR
BG
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
COR
CNC
CNP
CF
CHRW
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CPAY
CTVA
CSGP
COST
CTRA
CRWD
CCI
CSX
CMI
CVS
DHR
DRI
DVA
DAY
DECK
DE
DAL
DVN
DXCM
FANG
DLR
DFS
DG
DLTR
D
DPZ
DOV
DOW
DHI
DTE
DUK
DD
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
LLY
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ESS
EL
ETSY
EG
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FIS
FITB
FSLR
FE
FI
FMC
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GE
GEHC
GEV
GEN
GNRC
GD
GIS
GM
GPC
GILD
GPN
GL
GDDY
GS
HAL
HIG
HAS
HCA
DOC
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON

### <font color='green'>Print the Data from news_table</font>

In [5]:
# Print the Data from news_table (optional)
# Example: Read one single day of headlines for ‘AMZN’ 
amzn = news_tables['AMZN']
# Get all the table rows tagged in HTML with <tr> into ‘amzn_tr’
amzn_tr = amzn.findAll('tr')
for i, table_row in enumerate(amzn_tr):
     # Read the text of the element ‘a’ into ‘link_text’
     a_text = table_row.a.text
     # Read the text of the element ‘td’ into ‘data_text’
     td_text = table_row.td.text
     # Print the contents of ‘link_text’ and ‘data_text’ 
     print(a_text)
     print(td_text)
     # Exit after printing 4 rows of data
     if i == 3:
         break

I Shopped on Temu: 3 Reasons I'm Not Convinced Temu Is a Big Threat to Amazon

            Today 11:22AM
        
2 Reasons to Keep an Eye on JD.com Stock

            11:08AM
        
Why Is Brad Gerstner Bullish on Amazon.com Inc (AMZN) Right Now?

            10:17AM
        
3 Artificial Intelligence (AI) Stocks to Buy Now and Hold for Decades

            10:04AM
        


### <font color='green'>Parse the Date, Time and News Headlines into a Python List</font>

In [6]:
# Parse the Date, Time and News Headlines into a Python List
parsed_news = []
# Iterate through the news
for file_name, news_table in news_tables.items():
    # Iterate through all tr tags in 'news_table'
    for x in news_table.findAll('tr'):
        # read the text from each tr tag into text
        # get text from a only
        if x.a: # Avoid Null value
            text = x.a.get_text()
        else:
            continue
        # splite text in the td tag into a list 
        date_scrape = x.td.text.split()
        # if the length of 'date_scrape' is 1, load 'time' as the only element
        if len(date_scrape) == 1:
            time = date_scrape[0]
            date = datetime.now().date()
        # else load 'date' as the 1st element and 'time' as the second    
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        # Extract the ticker from the file name, get the string up to the 1st '_'  
        ticker = file_name
        
        # Append ticker, date, time and headline as a list to the 'parsed_news' list
        parsed_news.append([ticker, date, time, text])
        
parsed_news[:5] # print first 5 rows of news

[['MMM',
  'Jun-27-24',
  '09:30AM',
  '3M Young Scientist Challenge Announces 2024 National Finalists'],
 ['MMM',
  datetime.date(2024, 6, 29),
  '09:00AM',
  'Is Trending Stock 3M Company (MMM) a Buy Now?'],
 ['MMM',
  'Jun-26-24',
  '09:30AM',
  '3M and Discovery Education Recognize 26 State Merit Winners and Four Honorable Mentions in 2024 3M Young Scientist Challenge'],
 ['MMM',
  'Jun-25-24',
  '05:45PM',
  '3M (MMM) Stock Falls Amid Market Uptick: What Investors Need to Know'],
 ['MMM',
  datetime.date(2024, 6, 29),
  '05:44PM',
  'Connecticut firefighters sue DuPont, 3M, Honeywell over allegedly contaminated gear']]

### <font color='green'>Perform Sentiment Analysis with Vader</font>

In [7]:
# Perform Sentiment Analysis with Vader
# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()
# Set column names
columns = ['ticker', 'date', 'time', 'headline']
# Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)

# Convert today to the date format
parsed_and_scored_news['date'] = parsed_and_scored_news['date'].apply(lambda x: datetime.now().date() if x == 'Today' else x)

# Iterate through the headlines and get the polarity scores using vader
scores = parsed_and_scored_news['headline'].apply(vader.polarity_scores).tolist()
# Convert the 'scores' list of dicts into a DataFrame
scores_df = pd.DataFrame(scores)

# Join the DataFrames of the news and the list of dicts
parsed_and_scored_news = parsed_and_scored_news.join(scores_df, rsuffix='_right')
# Convert the date column from string to datetime
parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date
parsed_and_scored_news.head()

# Make sure the compound lies between -1 to 1
print(parsed_and_scored_news[['neg', 'neu', 'pos', 'compound']].describe())

                neg           neu           pos      compound
count  49918.000000  49918.000000  49918.000000  49918.000000
mean       0.027786      0.860932      0.111282      0.142887
std        0.078235      0.161661      0.148080      0.292707
min        0.000000      0.201000      0.000000     -0.926000
25%        0.000000      0.741000      0.000000      0.000000
50%        0.000000      1.000000      0.000000      0.000000
75%        0.000000      1.000000      0.217000      0.340000
max        0.667000      1.000000      0.787000      0.960900


In [122]:
# Visualize the data
parsed_and_scored_news

,ticker,date,time,headline,neg,neu,pos,compound
0,MMM,2024-06-25,05:45PM,3M (MMM) Stock Falls Amid Market Uptick: What ...,0.000,1.000,0.000,0.0000
1,MMM,2024-06-25,05:44PM,"Connecticut firefighters sue DuPont, 3M, Honey...",0.000,1.000,0.000,0.0000
2,MMM,2024-06-25,03:00PM,3 Stocks to Buy Before the Expected Rate Cut T...,0.189,0.811,0.000,-0.2732
3,MMM,2024-06-21,12:14PM,The 30-stock secret: Dont fight Papa Dow,0.000,0.733,0.267,0.2924
4,MMM,2024-06-19,10:11PM,"Zeek, a New Decentralized Social Collaboration...",0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...
49910,ZTS,2024-03-27,04:26PM,Largest Animal Health Firm Zoetis Hit With Eur...,0.148,0.852,0.000,-0.5106
49911,ZTS,2024-03-26,10:27AM,UPDATE 2-Zoetis in EU antitrust crosshairs ove...,0.248,0.752,0.000,-0.5106
49912,ZTS,2024-06-25,09:34AM,Zoetis in EU antitrust crosshairs over rival d...,0.268,0.732,0.000,-0.5106
49913,ZTS,2024-06-25,09:30AM,EU targets Zoetis in antitrust investigation o...,0.000,1.000,0.000,0.0000


### <font color='green'>Calculate Mean Sentiment for Each Ticker</font>

In [8]:
# Group by each ticker and get the mean of all sentiment scores
mean_scores = parsed_and_scored_news.groupby(['ticker']).mean()
mean_scores

/var/folders/7k/vccv80yd3nsdc972smlpyx780000gn/T/ipykernel_44927/2723772882.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mean_scores = parsed_and_scored_news.groupby(['ticker']).mean()


,neg,neu,pos,compound
ticker,,,,
A,0.01648,0.88186,0.10166,0.141734
AAL,0.10006,0.79991,0.10002,0.000527
AAPL,0.03831,0.83068,0.13100,0.183216
ABBV,0.03019,0.83570,0.13410,0.179221
ABNB,0.03938,0.82742,0.13319,0.157043
...,...,...,...,...
XYL,0.00647,0.87191,0.12163,0.184680
YUM,0.01760,0.89278,0.08961,0.139354
ZBH,0.00922,0.91351,0.07726,0.125536


### <font color='green'>Get Current Price, Sector and Industry of each Ticker</font>

In [9]:
# Fetch additional information from Yahoo Finance
sectors = []
industries = []
prices = []
for ticker in tickers:
    print(ticker)
    tickerdata = yf.Ticker(ticker)
    try:
        prices.append(tickerdata.info['currentPrice'])
        sectors.append(tickerdata.info['sector'])
        industries.append(tickerdata.info['industry'])
    except KeyError:
        prices.append(None)
        sectors.append(None)
        industries.append(None)

MMM
AOS
ABT
ABBV
ACN
ADBE
AMD
AES
AFL
A
APD
ABNB
AKAM
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
AME
AMGN
APH
ADI
ANSS
AON
APA
AAPL
AMAT
APTV
ACGL
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BK
BBWI
BAX
BDX
BRK.B
BBY
BIO
TECH
BIIB
BLK
BX
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BRO
BF.B
BLDR
BG
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
COR
CNC
CNP
CF
CHRW
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CPAY
CTVA
CSGP
COST
CTRA
CRWD
CCI
CSX
CMI
CVS
DHR
DRI
DVA
DAY
DECK
DE
DAL
DVN
DXCM
FANG
DLR
DFS
DG
DLTR
D
DPZ
DOV
DOW
DHI
DTE
DUK
DD
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
LLY
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ESS
EL
ETSY
EG
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FIS
FITB
FSLR
FE
FI
FMC
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GE
GEHC
GEV
GEN
GNRC
GD
GIS
GM
GPC
GILD
GPN
GL
GDDY
GS
HAL
HIG
HAS
HCA
DOC
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HWM

### <font color='green'>Combine the Information Above and the Corresponding Tickers into a DataFrame</font>

In [10]:
# Create dataframe with additional information
d = {'Sector': sectors, 'Industry': industries, 'Price': prices}
df_info = pd.DataFrame(data=d, index=tickers)

### <font color='green'>Join all the Information into a Single DataFrame</font>

In [11]:
# Merge sentiment scores and stock information
df = mean_scores.join(df_info)
df = df.rename(columns={"compound": "Sentiment Score", "neg": "Negative", "neu": "Neutral", "pos": "Positive"})
df = df.reset_index()
df

,ticker,Negative,Neutral,Positive,Sentiment Score,Sector,Industry,Price
0,A,0.01648,0.88186,0.10166,0.141734,Healthcare,Diagnostics & Research,129.63
1,AAL,0.10006,0.79991,0.10002,0.000527,Industrials,Airlines,11.33
2,AAPL,0.03831,0.83068,0.13100,0.183216,Technology,Consumer Electronics,210.62
3,ABBV,0.03019,0.83570,0.13410,0.179221,Healthcare,Drug Manufacturers - General,171.52
4,ABNB,0.03938,0.82742,0.13319,0.157043,Consumer Cyclical,Travel Services,151.63
...,...,...,...,...,...,...,...,...
496,XYL,0.00647,0.87191,0.12163,0.184680,Industrials,Specialty Industrial Machinery,135.63
497,YUM,0.01760,0.89278,0.08961,0.139354,Consumer Cyclical,Restaurants,132.46
498,ZBH,0.00922,0.91351,0.07726,0.125536,Healthcare,Medical Devices,108.53
499,ZBRA,0.00899,0.89448,0.09653,0.146205,Technology,Communication Equipment,308.93


### <font color='green'>Choose between Most Positive or Most Negative Stocks' Sentiment Score</font>

In [21]:
# Define the function to filter sentiment scores
def filter_tickers(df, choice, num_of_company=5):
    filtered_df = pd.DataFrame()
    sectors = df['Sector'].unique()
    
    for sector in sectors:
        sector_df = df[df['Sector'] == sector]
        if choice == 'most positive':
            top_stocks = sector_df.nlargest(num_of_company, 'Sentiment Score')
        elif choice == 'most negative':
            top_stocks = sector_df.nsmallest(num_of_company, 'Sentiment Score')
        filtered_df = pd.concat([filtered_df, top_stocks])
    
    return filtered_df

# Choose between most positive and most negative
choice = 'most negative' 
filtered_df = filter_tickers(df, choice)
filtered_df = filtered_df.reset_index(drop=True)

### The Final Treemap

In [22]:
# Create a treemap (No need to know how large each of ticker is in our position)
fig = px.treemap(filtered_df, path=[px.Constant("Sectors"), 'Sector', 'Industry', 'ticker'],
                  color='Sentiment Score', hover_data=['Price', 'Negative', 'Neutral', 'Positive', 'Sentiment Score'],
                  color_continuous_scale=['#FF0000', "#000000", '#00FF00'],
                  color_continuous_midpoint=0)

fig.data[0].customdata = filtered_df[['Price', 'Negative', 'Neutral', 'Positive', 'Sentiment Score']].round(3) # round to 3 decimal places
fig.data[0].texttemplate = "%{label}<br>%{customdata[4]}"

fig.update_traces(textposition="middle center")
fig.update_layout(margin=dict(t=30, l=10, r=10, b=10), font_size=20)

plotly.offline.plot(fig, filename='stock_sentiment.html') # this writes the plot into a html file and opens it
fig.show()